In [ ]:
import csv
import os
import numpy as np
from skimage.transform import resize
import matplotlib.pyplot as plt
import skimage.feature
import skimage.segmentation
import cv2
from skimage import img_as_float32, img_as_ubyte

def isFloat(x):
    try:
        float(x)
        return True
    except:
        return False
        
def pickmatrix(matirx):
    idx_row = np.argwhere(np.all(matirx[...,:] == 0, axis = 0))
    matirx = np.delete(matirx, idx_row, axis = 1)
    idx2_col = np.argwhere(np.all(matirx[...,:] == 0, axis = 1))
    matirx = np.delete(matirx, idx2_col, axis = 0)
    H, W, C = matirx.shape
    return matirx[(H//2 - 40):(H//2 + 40), (W//2 - 40):(W//2 + 40), :]

def load_csv(filepath):
    CSVfiles = sorted(os.listdir(filepath))
    # print(CSVfiles)
    for file in CSVfiles:
        a = file.split('_',10)
        if len(a) > 4:
            if a[5] == 'CUR.CSV':
                cur_file = os.path.join(filepath, file)
            if a[5] == 'PAC.CSV':
                pac_file = os.path.join(filepath, file)

    curfile = open(cur_file, "r", encoding='unicode_escape')
    curreader = csv.reader(curfile)
    pacfile = open(pac_file, "r", encoding='unicode_escape')
    pacreader = csv.reader(pacfile)

    total_matrix = np.zeros((141,141,4))
    mt_number = 3
    mt_row = 0
    mt = np.zeros((80,80,3))
    for item in curreader:
        if len(item) == 1 or len(item) == 3:
            data = np.array(item[0].split(';'))  
            if data[0] in ['TANGENTIAL FRONT', 'TANGENTIAL BACK', 'TOTAL CORNEAL REFRACTIVE POWER']:
                mt_row = 0
                if data[0] == 'TANGENTIAL FRONT': mt_number = 0 
                if data[0] == 'TANGENTIAL BACK': mt_number = 1
                if data[0] == 'TOTAL CORNEAL REFRACTIVE POWER': mt_number = 3
            if isFloat(data[0]) is True:
                data[data==''] = '0'
                total_matrix[int(-10*float(data[0])+70),:,mt_number] = data[1:142].astype(float)
                mt_row += 1
            if data[0] == '[SYSTEM]':  
                break
    total_matrix[total_matrix > 15] = 0
    for item in pacreader:
        if len(item) == 1:
            data = np.array(item[0].split(';')) 
            if data[1] == '-7.000':
                mt_row = 0
                mt_number = 2
            if isFloat(data[0]) is True:
                data[data==''] = '0'
                total_matrix[int(-10*float(data[0])+70),:,mt_number] = data[1:142].astype(float)
                mt_row += 1
            if data[0] == '[SYSTEM]':
                break    
    
    mt = pickmatrix(total_matrix[:,:,0:3])
    # print(mt[:,:,0])
    mt[:,:,0] = (mt[:,:,0] - 7) / (12 - 7)
    mt[:,:,1] = (mt[:,:,1] - 5) / (15 - 5)
    mt[:,:,2] = (mt[:,:,2] - 500) / (900 - 500)
    mt = resize(mt , (128,128))
    mt[mt < 0] = 0
    return mt

In [ ]:
baseline_file = '/data1/rong/code/Eyes/data/按月整理好的汇总/base/OKB0054_OD'
cure_file = '/data1/rong/code/Eyes/data/按月整理好的汇总/5/OKB0054_OD'
baseline = load_csv(baseline_file)
sixmonths = load_csv(cure_file)
# change = sixmonths[:,:,0] - baseline[:,:,0]
# change = (change - change.min()) / (change.max() - change.min())
# print(change)
# plt.figure()
# plt.imshow(change)
# plt.show()

In [ ]:
input_img = baseline[...,0]
input_img_ = np.clip(input_img, 0, 1)
why = img_as_ubyte(input_img_)

In [ ]:
import numpy as np

a = range(0, 8)
b = np.array(a)
c = b[np.newaxis, :]
print(np.arange(12))

i_index = np.repeat(c.T, 8, axis=1)
j_index = np.repeat(c, 8, axis=0)

qw = np.zeros((8,8,4))
qw[np.logical_and(i_index == 0, j_index == 1), 0] = 1

print(i_index)
print(j_index)

# print(qw)

In [ ]:
def gen_lens(r_basic, T_basic, r_reverse, T_reverse, r_pos, T_pos, r_side, T_side):

    pix = 1024
    total_length = 2*(r_basic + r_reverse + r_pos + r_side)
    pix_of_per_length = pix / total_length
    lens = np.zeros((pix, pix, 4))

    order = np.arange(pix)[np.newaxis, :]

    i_index = np.repeat(order.T, 1024, axis=1)
    j_index = np.repeat(order, 1024, axis=0)

    index = np.concatenate((i_index[:,:,np.newaxis], j_index[:,:,np.newaxis]), axis=2)
    center_index = np.array([(pix - 1)/2, (pix - 1)/2])[np.newaxis, np.newaxis, :]

    r_table  = np.linalg.norm(index - center_index, ord=2, axis=2) / pix_of_per_length

    lens[r_table < r_basic,0] = T_basic
    lens[np.logical_and(r_table >= r_basic, r_table < r_basic + r_reverse), 1] = T_reverse
    lens[np.logical_and(r_table >= r_basic + r_reverse,
     r_table < r_basic + r_reverse + r_pos), 2] = T_pos
    lens[np.logical_and(r_table >= r_basic + r_reverse + r_pos,
     r_table < r_basic + r_reverse + r_pos + r_side), 3] = T_side
    
    return lens, pix_of_per_length

In [ ]:
import matplotlib.pyplot as plt

len_test, _ = gen_lens(0.3, 0.2, 0.1, 0.4, 0.2, 0.6, 0.1, 0.8)

rainbow = plt.get_cmap('rainbow')

plt.figure(1)
plt.imshow(rainbow(len_test[:,:,0]), cmap=rainbow)
plt.colorbar()
plt.show()


plt.figure(2)
plt.imshow(rainbow(len_test[:,:,1]), cmap=rainbow)
plt.colorbar()
plt.show()

plt.figure(3)
plt.imshow(rainbow(len_test[:,:,2]), cmap=rainbow)
plt.colorbar()
plt.show()

plt.figure(4)
plt.imshow(rainbow(len_test[:,:,3]), cmap=rainbow)
plt.colorbar()
plt.show()

merge = len_test.sum(axis=2)

plt.figure(5)
plt.imshow(rainbow(np.asarray(merge==0, dtype=np.float64)), cmap=rainbow)
plt.colorbar()
plt.show()

In [ ]:
a = np.random.rand(1,4)
print(a)
print(a[0,-1:])
# a.sum(axis=2, keepdims=True)

In [ ]:
import numpy as np
import os

np.random.seed(1234)

with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/爱尔按月整理好/train.txt', 'w') as f1:
    with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/爱尔按月整理好/val.txt', 'w') as f2:
        with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/爱尔按月整理好/爱尔全部数据.txt', 'r') as f:
            for line in f:
                if np.random.rand() < 0.8:
                    f1.write(line)
                else:
                    f2.write(line)


In [ ]:
import numpy as np
import os

np.random.seed(1234)

with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/train.txt', 'w') as f1:
    with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/val.txt', 'w') as f2:
        with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/alldata.txt', 'r') as f:
            for line in f:
                if np.random.rand() < 0.8:
                    f1.write(line)
                else:
                    f2.write(line)

In [ ]:
baseline_files = []
lines = []
csv_dir = '/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/爱尔按月整理好'
f = open(csv_dir + '/train.txt', 'r')
for line in f:
    lines.append(line)
    line = line.strip()
    line = line.split(',')
    baseline_files.append([line[0],line[1]])        

baseline_filenames = [(os.path.join(csv_dir, 'base', x[1]), os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]
# sixmonths_filenames = [(os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]

with open(csv_dir + '/train_1.txt', 'w') as f:
    for asd in baseline_filenames:
        file1 = asd[0]
        file2 = asd[1]
        line = asd[2]
        CSVfiles = sorted(os.listdir(file1))
        # print(CSVfiles)
        # print(CSVfiles)
        flag1 = False
        flag2 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag1 = True
                if a[5] == 'PAC.CSV':
                    flag2 = True
        
        CSVfiles = sorted(os.listdir(file2))
        # print(CSVfiles)
        # print(CSVfiles)
        flag3 = False
        flag4 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag3 = True
                if a[5] == 'PAC.CSV':
                    flag4 = True
        
        if flag1 and flag2 and flag3 and flag4:
            f.write(line)

In [ ]:
baseline_files = []
lines = []
csv_dir = '/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/爱尔按月整理好'
f = open(csv_dir + '/val.txt', 'r')
for line in f:
    lines.append(line)
    line = line.strip()
    line = line.split(',')
    baseline_files.append([line[0],line[1]])        

baseline_filenames = [(os.path.join(csv_dir, 'base', x[1]), os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]
# sixmonths_filenames = [(os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]

with open(csv_dir + '/val_1.txt', 'w') as f:
    for asd in baseline_filenames:
        file1 = asd[0]
        file2 = asd[1]
        line = asd[2]
        CSVfiles = sorted(os.listdir(file1))
        # print(CSVfiles)
        # print(CSVfiles)
        flag1 = False
        flag2 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag1 = True
                if a[5] == 'PAC.CSV':
                    flag2 = True
        
        CSVfiles = sorted(os.listdir(file2))
        # print(CSVfiles)
        # print(CSVfiles)
        flag3 = False
        flag4 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag3 = True
                if a[5] == 'PAC.CSV':
                    flag4 = True
        
        if flag1 and flag2 and flag3 and flag4:
            f.write(line)

In [ ]:
baseline_files = []
lines = []
csv_dir = '/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据'
f = open(csv_dir + '/train.txt', 'r')
for line in f:
    lines.append(line)
    line = line.strip()
    line = line.split(',')
    baseline_files.append([line[0],line[1]])        

baseline_filenames = [(os.path.join(csv_dir, 'base', x[1]), lines[i]) for i, x in enumerate(baseline_files)]
sixmonths_filenames = [(os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]

with open(csv_dir + '/train_1.txt', 'w') as f:
    for asd in baseline_filenames:
        zxc = asd[0]
        line = asd[1]
        CSVfiles = sorted(os.listdir(zxc))
        # print(CSVfiles)
        # print(CSVfiles)
        flag1 = False
        flag2 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag1 = True
                if a[5] == 'PAC.CSV':
                    flag2 = True
        
        if flag1 and flag2:
            f.write(line)

In [ ]:
baseline_files = []
lines = []
csv_dir = '/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据'
f = open(csv_dir + '/val.txt', 'r')
for line in f:
    lines.append(line)
    line = line.strip()
    line = line.split(',')
    baseline_files.append([line[0],line[1]])        

baseline_filenames = [(os.path.join(csv_dir, 'base', x[1]), lines[i]) for i, x in enumerate(baseline_files)]
sixmonths_filenames = [(os.path.join(csv_dir, x[0], x[1]), lines[i]) for i, x in enumerate(baseline_files)]

with open(csv_dir + '/val_1.txt', 'w') as f:
    for asd in baseline_filenames:
        zxc = asd[0]
        line = asd[1]
        CSVfiles = sorted(os.listdir(zxc))
        # print(CSVfiles)
        # print(CSVfiles)
        flag1 = False
        flag2 = False
        for file in CSVfiles:
            a = file.split('_',10)
            if len(a) > 4:
                if a[5] == 'CUR.CSV':
                    flag1 = True
                if a[5] == 'PAC.CSV':
                    flag2 = True
        
        if flag1 and flag2:
            f.write(line)

In [ ]:
import openpyxl
import os
import numpy as np

path = '/data1/rong/code/Eyes/data/lens_paras.xlsx'
workbook = openpyxl.load_workbook(path)
M_len = workbook['M_401']
M = M_len.values
E_len = workbook['E_212']
Index = workbook['Index']

tmp = []
j = 0
for c, i in enumerate(M):
    j = j + 1
    tmp.append(i)
    if j == 4: break
    print(c)

for i in range(74):
    print(i, tmp[0][i], tmp[1][i], tmp[2][i], tmp[3][i])
    print(i, tmp[0][i].__class__, tmp[1][i].__class__, tmp[2][i].__class__, tmp[3][i].__class__)

In [ ]:
import openpyxl
import os
import numpy as np

path = '/data1/rong/code/Eyes/data/lens_paras.xlsx'
workbook = openpyxl.load_workbook(path)
M_len_sheet = workbook['M_401']
E_len_sheet = workbook['E_212']
Index_sheet = workbook['Index']
m_len = []
m_index = []
e_len = []
e_index = []
index = {}

M_stop = 339

for i in Index_sheet.values:
    index[i[0]] = i[1]
# E, F, G, H, I, J, K
# 定位弧曲率	降幅	总直径	光学区直径	反转弧宽度	定位弧宽度	周边弧宽度
for _, i in enumerate(M_len_sheet.values):
    m_index.append('{}_OD'.format(index[i[0]]))
    m_index.append('{}_OS'.format(index[i[0]]))
    m_len.append(i[4:11])
    m_len.append(i[11:18])
# E, F, G, H, I, J, K, L
# 定位弧AC1曲率	降幅	总直径	光学区直径	反转弧宽度	定位弧AC1宽度	定位弧AC2宽度	周边弧宽度
for _, i in enumerate(E_len_sheet.values):
    e_index.append('{}_OD'.format(index[i[0]]))
    e_index.append('{}_OS'.format(index[i[0]]))
    e_len.append(i[4:12])
    e_len.append(i[12:20])

m_len_clean = []
m_index_clean = []
e_len_clean = []
e_index_clean = []

# E, F, G, H, I, J, K
# 定位弧曲率	降幅	总直径	光学区直径	反转弧宽度	定位弧宽度	周边弧宽度
for i, item in enumerate(m_len[0:(2*M_stop + 1)]):
    if (isinstance(item[-1], float) or isinstance(item[-1], int)) and (isinstance(item[-2], float) or isinstance(item[-2], int)):
        m_index_clean.append(m_index[i])
        m_len_clean.append(item)

# E, F, G, H, I, J, K, L
# 定位弧AC1曲率	降幅	总直径	光学区直径	反转弧宽度	定位弧AC1宽度	定位弧AC2宽度	周边弧宽度
for i, item in enumerate(e_len):
    if (isinstance(item[-1], float) or isinstance(item[-1], int)) and (isinstance(item[1], float) or isinstance(item[1], int)):
        e_index_clean.append(e_index[i])
        e_len_clean.append(item)

In [ ]:
m_test = np.array(m_len_clean, dtype=float)
e_test = np.array(e_len_clean, dtype=float)

print(m_test.mean(axis=0))
print(e_test.mean(axis=0))

In [ ]:
import os
with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/len_record_all.txt', 'r') as f:
    for file in f:
        file = file.strip()
        print('\n', file.split('/')[-3])
        with open(file, 'r') as fin:
            for j in fin:
                print(j.strip())

In [ ]:
import os
have_lens = []
with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/len_record.txt', 'r') as f:
    for file in f:
        file = file.strip()
        have_lens.append(file.split('/')[-3])

have_lens = set(have_lens)

src_txts_pre = os.listdir('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/txts/src')
src_txts = []
for i in src_txts_pre:

    if '_with_lens.txt' not in i:

        src_txts.append(os.path.join('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/txts/src', i))

for txt in src_txts:

    fout = open(txt.replace('.txt', '_with_lens.txt'), 'w')
    with open(txt, 'r') as f:
        for line in f:
            line_ = line.strip()
            name = line_.split(',')[1]
            if name in have_lens:
                fout.write(line)
    
    fout.close()

In [ ]:
import os
have_lens = []
with open('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/len_record_all.txt', 'r') as f:
    for file in f:
        file = file.strip()
        have_lens.append(file.split('/')[-3])

have_lens = set(have_lens)

src_txts_pre = os.listdir('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/txts/src')
src_txts = []
for i in src_txts_pre:

    if '_with_lens.txt' not in i and '.txt' in i:

        src_txts.append(os.path.join('/data1/rong/code/Eyes/data/按月整理的角膜地形图数据/所有按月整理的数据/txts/src', i))

for txt in src_txts:

    fout = open(txt.replace('.txt', '_with_lens.txt'), 'w')
    with open(txt, 'r') as f:
        for line in f:
            line_ = line.strip()
            name = line_.split(',')[1]
            if name in have_lens:
                fout.write(line)
    
    fout.close()

In [ ]:
filename = '/data1/rong/code/Eyes/exps/_base_no_LBPorLEN/2022-12-02T01:33:31.711394/opt_record.txt'
info={}
with open(filename, 'r') as f:
    for line in f:
        line_ = line.strip()
        tmp = line_.split(':')
        if len(tmp) == 2:
            info[tmp[0]] = tmp[1]

print(info)

In [ ]:
import torch


In [20]:

a = torch.tensor([[1.0,2],[3,4]])
b = a==2
print(b.float())

tensor([[0., 1.],
        [0., 0.]])


In [ ]:
import os
for i in range(25):
    command = 'cp -r /data1/rong/code/Eyes/data/_src/spar_aier/{}/* /data1/rong/code/Eyes/data/spar/{}/'.format(i, i)
    print(command)
    os.system(command)

In [3]:
import torch
a = torch.rand((2,5,1,4))
b = torch.rand((2,1,5,4))
c = a-b
d = torch.norm(a-b, dim=-1)
print(c.shape)
print(d.shape)
# print()

torch.Size([2, 5, 5, 4])
torch.Size([2, 5, 5])


In [1]:
from skimage.util import img_as_ubyte
import numpy as np

a = np.array([[0.5,0.6],[1.2,2]])
b = img_as_ubyte(a)
print(b)

ValueError: Images of type float must be between -1 and 1.

In [2]:
from sklearn.metrics import mean_squared_error
import numpy as np

a = np.array([1])
b = np.array([0.5])
c = mean_squared_error(a,b)
print(c)

0.25
